In [1]:
## Project 2 Airbnb Analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
import glob

import plotly
import plotly.plotly as py
py.sign_in('nzahadat', 'VDBLh2LQZvNWxovGAbr8')
print(plotly.__version__)
plotly.offline.init_notebook_mode() # run at the start of every notebook


%matplotlib inline

2.0.11


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [52]:
total_listings=pd.DataFrame()
for filename in glob.iglob('./**/**/listings.csv',recursive=True):
    print(filename)
    data=pd.read_csv(filename,error_bad_lines=False,low_memory=False,encoding='ISO-8859-1')
    total_listings=total_listings.append(data)
    
    

./2015-01-01/data/listings.csv
./2015-03-01/data/listings.csv
./2015-04-01/data/listings.csv
./2015-05-01/data/listings.csv
./2015-06-01/data/listings.csv
./2015-08-01/data/listings.csv
./2015-09-01/data/listings.csv
./2015-10-01/data/listings.csv
./2015-11-01/data/listings.csv
./2015-11-20/data/listings.csv
./2015-12-02/data/listings.csv
./2016-01-01/data/listings.csv
./2016-02-02/data/listings.csv
./2016-04-03/data/listings.csv
./2016-05-02/data/listings.csv
./2016-06-02/data/listings.csv
./2016-07-02/data/listings.csv
./2016-08-02/data/listings.csv
./2016-09-02/data/listings.csv
./2016-10-01/data/listings.csv
./2016-11-02/data/listings.csv
./2016-12-03/data/listings.csv
./2017-01-01/data/listings.csv
./2017-02-02/data/listings.csv
./2017-03-02/data/listings.csv
./2017-04-02/data/listings.csv
./2017-05-02/data/listings.csv
./2015-01-01/data/listings.csv


KeyboardInterrupt: 

In [ ]:
#list(total_listings)
total_listings.summary()
